# Assignment 6 - Dialog Systems and Question Answering

- Student 1 - Luca Modica
- Student 2 - Hugo Alves Henriques E Silva

---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from icecream import ic


sns.set_style()
%matplotlib inline
np.random.seed(0)

Example from chatGPT

In [ ]:
import requests

# API endpoints
weather_api_endpoint = "https://api.weather.com/forecast"
restaurant_api_endpoint = "https://api.yelp.com/v3/businesses/search"
transportation_api_endpoint = "https://api.transportation.com/schedule"

# API keys
weather_api_key = "your_weather_api_key"
restaurant_api_key = "your_restaurant_api_key"
transportation_api_key = "your_transportation_api_key"

def get_weather_forecast(city):
    params = {
        "apikey": weather_api_key,
        "city": city,
        "days": 1
    }
    response = requests.get(weather_api_endpoint, params=params)
    data = response.json()
    forecast = data["forecast"]["daily"]["summary"]
    return forecast

def find_restaurant(cuisine):
    headers = {
        "Authorization": f"Bearer {restaurant_api_key}"
    }
    params = {
        "term": cuisine,
        "location": "your_location"
    }
    response = requests.get(restaurant_api_endpoint, headers=headers, params=params)
    data = response.json()
    businesses = data["businesses"]
    if businesses:
        return random.choice(businesses)["name"]
    else:
        return None

def get_next_tram_bus(mode):
    headers = {
        "Authorization": f"Bearer {transportation_api_key}"
    }
    params = {
        "mode": mode,
        "location": "your_location"
    }
    response = requests.get(transportation_api_endpoint, headers=headers, params=params)
    data = response.json()
    schedule = data["schedule"]
    if schedule:
        return random.choice(schedule)["time"]
    else:
        return None

# Rest of the code remains the same as the previous implementation